This project is aimed at modifying Yan Gobeil's code in order to generate passwords using a Recurrent Neural Network. I've provided a list of around one million passwords for the computer to train on in the hope that it can provide us with a model that somewhat accurately predicts passwords in the future.

I'm new here, so if my code sucks, I'm sorry, but I'm still learning. If you have ideas on how I could improve it I'd love it if you would share them with me.

Without Yan's prior work I wouldn't have been able to do this, so thank you sir!

Here's a link to the man's github:
https://github.com/yangobeil/

Now without further adoo, lets get to it!

BEFORE YOU START: DOWNLOAD rockyou.txt from http://downloads.skullsecurity.org/passwords/rockyou.txt.bz2
DON'T FORGET TO UNZIP IN THE DIRECTORY OF THIS FILE


# Gather and clean up the  data

In [1]:
# The first thing that needs to be done is to import all the modules that we need for the cleaning stage:

SyntaxError: invalid syntax (<ipython-input-1-f9ebbc247f6b>, line 1)

In [1]:
from collections import Counter
import numpy as np
import requests
import random
import os
from keras.models import load_model

Using TensorFlow backend.


The next section imports the password list that the network will train on

In [2]:

# passwords = []
# with open("./PWList.txt") as f:
#     for line in f.readlines():
#         if len(line) <= 10:
#             passwords.append(line.rstrip("\n"))


# print(passwords[:10]

# print(len(passwords))

In [3]:
# # because my computer wasn't able to load all of the list into ram while still being able to go through the entire process I decided to randomly pick half a million passwords at random, save that as a new list, and use that for the remainder of the program.
# passwords = random.sample(passwords, 500000)

In [4]:
# #Here the randomly picked training passwords are saved to a list to be used:

# with open('halfmil.txt','w') as f:
#   f.write('\n'.join(passwords))


In [5]:
# import the password training set

passwords = []
with open("./halfmil.txt") as f:
    for line in f.readlines():
        if len(line) <= 10:
            passwords.append(line.rstrip("\n"))

In [6]:
# here we get the model's vocabulary to get all the characters used in the training set

chars_in_traininglist = []
for pw in passwords:
    for c in pw:
        chars_in_traininglist.append(c)

In [7]:
print(set(chars_in_traininglist))
print(len(set(chars_in_traininglist)))

{'o', 'R', 'D', 'K', '|', ';', 'f', 'Y', 'G', 'l', 'i', 'j', 't', '!', '?', ']', 'w', 'y', '$', 'z', "'", 'e', 'E', 'n', 'V', '`', 'F', '[', '0', '~', '6', '7', 'q', 'W', 'k', 'H', 'M', '-', 'Q', '.', '(', '4', 'J', 's', 'N', '/', 'C', 'p', 'Z', 'I', 'c', '_', '%', 'm', 'L', '\\', 'O', 'r', '^', 'g', '#', '&', '8', '{', 'b', '3', 'U', '1', '9', 'A', '+', 'h', 'S', '5', 'B', 'T', '@', '2', 'd', 'u', 'v', 'a', 'x', 'P', 'X', '*'}
86


In [8]:
# if the need exists for a smaller vocabulary due to hardware limitations you can scan for character which only appear once or twice in the training set. This will decrease the size of each array created. This can give an idea on which characters could be excluded.

freq_ch_traininglist = Counter()
for i in chars_in_traininglist:
    for ch in i:
        freq_ch_traininglist[ch] +=1
        
print(freq_ch_traininglist.most_common())

[('a', 217243),
 ('1', 205747),
 ('e', 183494),
 ('0', 147785),
 ('o', 144080),
 ('2', 140743),
 ('r', 134642),
 ('i', 131618),
 ('n', 126494),
 ('s', 117276),
 ('9', 114097),
 ('l', 103881),
 ('t', 103664),
 ('3', 96078),
 ('5', 88268),
 ('7', 86532),
 ('6', 84650),
 ('8', 83419),
 ('4', 82429),
 ('m', 80667),
 ('d', 77991),
 ('c', 70715),
 ('y', 70388),
 ('u', 66701),
 ('b', 64587),
 ('h', 64558),
 ('k', 63081),
 ('w', 58489),
 ('g', 55779),
 ('p', 51320),
 ('z', 47722),
 ('f', 45568),
 ('v', 43549),
 ('x', 36874),
 ('j', 31724),
 ('q', 12597),
 ('A', 10116),
 ('W', 9563),
 ('S', 8376),
 ('Y', 8280),
 ('E', 8267),
 ('Z', 8052),
 ('R', 7639),
 ('X', 7040),
 ('T', 6998),
 ('N', 6691),
 ('L', 6617),
 ('B', 6572),
 ('M', 6497),
 ('C', 6235),
 ('D', 6197),
 ('V', 5907),
 ('P', 5876),
 ('O', 5857),
 ('I', 5528),
 ('H', 5280),
 ('G', 5027),
 ('K', 4909),
 ('F', 4721),
 ('U', 4571),
 ('J', 4096),
 ('Q', 2655),
 ('_', 734),
 ('.', 663),
 ('-', 503),
 ('!', 333),
 ('*', 321),
 ('@', 200),
 ('$

In [9]:
further refinement of vocabulary

extended_ascii = [chr(i) for i in range(32,256)]

custom_ascii = []
for ch in set(extended_ascii):
    if ch in set(chars_in_traininglist):
        custom_ascii.append(ch)
custom_ascii = sorted(custom_ascii)

print(custom_ascii)
print(len(custom_ascii))


['!', '#', '$', '%', '&', "'", '(', '*', '+', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~']
86


In [10]:
# Here you can remove uncommon characters from your trainingset if its nessecary to shrink the array

exclusion = [')','¼','é','»','ª','}']
for i in custom_ascii:
    if i in exclusion:
        custom_ascii.remove(i)

print(len(exclusion))
print(len(custom_ascii))
print(custom_ascii)



6
86
['!', '#', '$', '%', '&', "'", '(', '*', '+', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~']


In [11]:
# When making use of  the above cell also make sure that the entries in the training set which contain the excluded characters are also removed.

for ch in exclusion:
    for i in passwords:
        if ch in i:
            passwords.remove(i)

# Print out the new length of the adjusted trainingset
print(len(passwords))

500000


In [12]:
#amount of characters in trainingset: (for use later in character indexing)
print(len(set(chars_in_traininglist)))

{'o', 'R', 'D', 'K', '|', ';', 'f', 'Y', 'G', 'l', 'i', 'j', 't', '!', '?', ']', 'w', 'y', '$', 'z', "'", 'e', 'E', 'n', 'V', '`', 'F', '[', '0', '~', '6', '7', 'q', 'W', 'k', 'H', 'M', '-', 'Q', '.', '(', '4', 'J', 's', 'N', '/', 'C', 'p', 'Z', 'I', 'c', '_', '%', 'm', 'L', '\\', 'O', 'r', '^', 'g', '#', '&', '8', '{', 'b', '3', 'U', '1', '9', 'A', '+', 'h', 'S', '5', 'B', 'T', '@', '2', 'd', 'u', 'v', 'a', 'x', 'P', 'X', '*'}
86


In [13]:
chars_in_traininglist = []
for pw in passwords:
    for c in pw:
        chars_in_traininglist.append(c)
print(set(chars_in_traininglist))
print(len(set(chars_in_traininglist)))

{'o', 'R', 'D', 'K', '|', ';', 'f', 'Y', 'G', 'l', 'i', 'j', 't', '!', '?', ']', 'w', 'y', '$', 'z', "'", 'e', 'E', 'n', 'V', '`', 'F', '[', '0', '~', '6', '7', 'q', 'W', 'k', 'H', 'M', '-', 'Q', '.', '(', '4', 'J', 's', 'N', '/', 'C', 'p', 'Z', 'I', 'c', '_', '%', 'm', 'L', '\\', 'O', 'r', '^', 'g', '#', '&', '8', '{', 'b', '3', 'U', '1', '9', 'A', '+', 'h', 'S', '5', 'B', 'T', '@', '2', 'd', 'u', 'v', 'a', 'x', 'P', 'X', '*'}
86


In [14]:
# Length of the longest password in the training set
print (len(max(passwords, key=len)))

9


We should add a ',' at the end of each password. This will be useful to tell the RNN that the password is at its end.

In [17]:
passwords = list(map(lambda s: s + ',', passwords ))
passwords [:10]

['arnasin,',
 'peregrina,',
 '777a77,',
 'Dfktynbyf,',
 '50055005,',
 'rachelm,',
 'zcfv131,',
 'since1989,',
 '03011970,',
 '3335577,']

# Transform the data

Now that we have our data cleaned up it's time to transform it into a form that the neural network will understand. The details of the model will be explained later but all we need to know is that we will input characters into the network instead of words. Each of these characters will then be converted to numbers and the conversion is done using the following dictionary:

In [18]:
# Convert from character to index

# Important: if you adjusted the trainingset to exclude uncommon characters be sure to adjust the char_to_index and index_to_char to the common 

char_to_index = dict((custom_ascii[i] , i) for i in range(0,len(set(chars_in_traininglist))))
char_to_index[','] = 86
print(char_to_index)

index_to_char = dict((i, custom_ascii[i]) for i in range(0,len(set(chars_in_traininglist))))
index_to_char[86] = ',' 
print(index_to_char)

{'!': 0, '#': 1, '$': 2, '%': 3, '&': 4, "'": 5, '(': 6, '*': 7, '+': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ';': 22, '?': 23, '@': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, '\\': 52, ']': 53, '^': 54, '_': 55, '`': 56, 'a': 57, 'b': 58, 'c': 59, 'd': 60, 'e': 61, 'f': 62, 'g': 63, 'h': 64, 'i': 65, 'j': 66, 'k': 67, 'l': 68, 'm': 69, 'n': 70, 'o': 71, 'p': 72, 'q': 73, 'r': 74, 's': 75, 't': 76, 'u': 77, 'v': 78, 'w': 79, 'x': 80, 'y': 81, 'z': 82, '{': 83, '|': 84, '~': 85, ',': 86}


{0: '!',
 1: '#',
 2: '$',
 3: '%',
 4: '&',
 5: "'",
 6: '(',
 7: '*',
 8: '+',
 9: '-',
 10: '.',
 11: '/',
 12: '0',
 13: '1',
 14: '2',
 15: '3',
 16: '4',
 17: '5',
 18: '6',
 19: '7',
 20: '8',
 21: '9',
 22: ';',
 23: '?',
 24: '@',
 25: 'A',
 26: 'B',
 27: 'C',
 28: 'D',
 29: 'E',
 30: 'F',
 31: 'G',
 32: 'H',
 33: 'I',
 34: 'J',
 35: 'K',
 36: 'L',
 37: 'M',
 38: 'N',
 39: 'O',
 40: 'P',
 41: 'Q',
 42: 'R',
 43: 'S',
 44: 'T',
 45: 'U',
 46: 'V',
 47: 'W',
 48: 'X',
 49: 'Y',
 50: 'Z',
 51: '[',
 52: '\\',
 53: ']',
 54: '^',
 55: '_',
 56: '`',
 57: 'a',
 58: 'b',
 59: 'c',
 60: 'd',
 61: 'e',
 62: 'f',
 63: 'g',
 64: 'h',
 65: 'i',
 66: 'j',
 67: 'k',
 68: 'l',
 69: 'm',
 70: 'n',
 71: 'o',
 72: 'p',
 73: 'q',
 74: 'r',
 75: 's',
 76: 't',
 77: 'u',
 78: 'v',
 79: 'w',
 80: 'x',
 81: 'y',
 82: 'z',
 83: '{',
 84: '|',
 85: '~',
 86: ','}

We need to define a few constants that will be useful later.

In [19]:
# maximum number of characters in passwords
# this will be the number of time steps in the RNN
max_char= len(max(passwords, key=len))
print(max_char)

# number of elements in the list of passwords, this is the number of training examples
m = len(passwords)
print(m)

# number of potential characters, this is the length of the input of each of the RNN units
char_dim = len(char_to_index)
print(char_dim)

10
500000
87


Finally we convert the list of passwords into a training dataset. The input X of the network is an array of size (m, max_char, char_dim). It contains a matrix for each of the m names. Each matrix contains a row for each character in the name. (Note that there are always the same number of rows and if the name doesn't have enough characters to fill the whole matrix the remaining rows contain nothing.) Each of these rows represents one character and it is encoded as a one-hot vector. This means that it is a vector of zeros with a one only in the entry that corresponds to the character that is present.

The output Y is the same as the input but translated by one unit. This means that the ith character in Y is the (i+1)th one in the actual password. This means that the network predicts the character that follows a given character in a sequence. 

In [20]:
X = np.zeros((m, max_char, char_dim))
Y = np.zeros((m, max_char, char_dim))

for i in range(m):
    pw = list(passwords[i])
    for j in range(len(pw)):
        X[i, j, char_to_index[pw[j]]] = 1
        if j < len(pw)-1:
            Y[i, j, char_to_index[pw[j+1]]] = 1

# RNN model

The model that we will use is a many-to-many recurrent neural network. This is a network that contains a given number of 'time' steps that each act with the same weights on the individual inputs and are all connected. Each time step takes in one input (in this case one character), and outputs a one-hot vector that represents the probabilities for the input of the next time step. 

In [21]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import LambdaCallback, ModelCheckpoint

In the case of interest here we only consider one layer of recurrence, which we take to be LSTM with 128 units. We return the output of this layer and use it into a fully connected dense layer that converts the result of the LSTM layer into a vector of size char_dim using a softmax activation. We use categorical cross entropy as a cost function because of the softmax result and use Adam optimization. There is not really any useful metric to judge if the model does good (in the future I might impliment a function where it queries the HIBP database to see if the password has occured in the wild, and if so to see how often, this might be included as feedback to tell the machine whether its doing good or not.) for now though we will mostly just look at the results.

In [22]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_char, char_dim), return_sequences=True))
model.add(Dense(char_dim, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

Once this model will be trained we will use it to create new passwords. This is achieved using the following function. The idea is to input empty characters to the trained network and use the output of the first time step as a probability distribution for the first letter of the name. We then use this distribution to decide randomly the first character, record it and update the input to pass this character as an input for the second time step. This is continued for the following time steps to create a password.

This is where using a ',' at the end of each name becomes important, because we stop the procedure once we get a ',' as an output, meaning that the generated password is done. Also if we reach the length of the largest name in the training set we put a ',' and end the procedure.

In [23]:
def make_pw(model):
    pw = []
    x = np.zeros((1, max_char, char_dim))
    end = False
    i = 0
    
    while end==False:
        probs = list(model.predict(x)[0,i])
        probs = probs / np.sum(probs)
        index = np.random.choice(range(char_dim), p=probs)
        if i == max_char-2:
            character = ','
            end = True
        else:
            character = index_to_char[index]
        pw.append(character)
        x[0, i+1, index] = 1
        i += 1
        if character == ',':
            end = True
    
    return ''.join(pw)

Now we want to use this function during the training to monitor how the generated passwords get better. To this end we create a function that will be given to the model when we fit it. We basically run the previous function a few times every epoch and print the results.

In [24]:
def generate_pw_loop(epoch, _):
    if epoch % 1 == 0:
        print('Passwords generated after epoch %d:' % epoch)

        for i in range(3):
            make_pw(model)

This converts the function to be able to use it in keras.

In [25]:
pw_generator = LambdaCallback(on_epoch_end = generate_pw_loop)

checkpoint_path = "./checkpoints/checkpoint-{epoch:03d}.h5"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# This saves the model each epoch
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    save_best_only=False,
    verbose=1,
    period=1)

We fit the model with the function and look at the results.

In [26]:
# We can now use the final trained model to generate names as we want.

NameError: name 'd' is not defined

In [27]:
def sample(model, nr_of_samples):
    sample_list = []

    for i in range(nr_of_samples):
        pw = make_pw(model)
        new_pw = pw[:-1]
        sample_list.append(new_pw)
    return sample_list

In [51]:
from tqdm import tqdm

def test_against_rockyou(model, times, nr_of_samples):
    # open rockyou list
    rockyou_list = []
    with open('./rockyou.txt',mode='r',encoding='latin-1') as f:
        for line in f.readlines():
            rockyou_list.append(line.rstrip("\n"))


    model_test_results = []
    for x in tqdm(range(times)):
        print("i")
        #sample 1000
        sample_list = sample(model, nr_of_samples)

        # test_against_rockyou
        rockyou_hits = 0
        for sample_item in sample_list:
            for rockyou_pw in rockyou_list:
                if sample_item == rockyou_pw:
                    rockyou_hits += 1

        model_test_results.append(rockyou_hits)
    return model_test_results

It could be possible to make the network better by changing the hyperparameters (number of units in LSTM layer, parameters of Adam optimization, adding extra LSTM layer) but I am not currently able to do that. One thing to notice is that there are some cases where the model seems to have overfitted and we recover known names or names very close to known ones.

In [32]:
from keras.models import load_model

# To load a previously saved model
# new_model = load_model("./checkpoints/checkpoint-010.h5")
# print("Model summary: \n\n")

# Start Training from scratch, or continue_training if you've got a checkpoint loaded
# new_model.fit(X, Y, batch_size=64, epochs=1, callbacks=[pw_generator, model_checkpoint_callback])

# Summarize model
# new_model.summary()

Model summary: 


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 128)           110592    
_________________________________________________________________
dense_1 (Dense)              (None, 10, 87)            11223     
Total params: 121,815
Trainable params: 121,815
Non-trainable params: 0
_________________________________________________________________


In [33]:
# prints a few examples for refference

# chose which model you want to load
new_model = load_model("./checkpoints/checkpoint-040.h5")

# print 5 new passwords:

run = sample(new_model, 5)
print(run)

['Delh', 'wattad', 'sa3beat', 'upgyj32', 'kareso']


In [77]:
# Test your generated passwords against the rockyou.txt list

runs = 3
new_model = load_model("./checkpoints/checkpoint-010.h5")

all = test_against_rockyou(new_model, runs ,100)
print(type(all))
total = 0
for i in all:
    print("this is i = "+str(i))
    total += i
    print(total)
print("average = " + str((total/10) / runs))







  0%|          | 0/5 [00:00<?, ?it/s]i




 20%|██        | 1/5 [00:03<00:14,  3.58s/it]i




 40%|████      | 2/5 [00:06<00:10,  3.52s/it]i




 60%|██████    | 3/5 [00:10<00:06,  3.46s/it]i




 80%|████████  | 4/5 [00:13<00:03,  3.43s/it]i




100%|██████████| 5/5 [00:16<00:00,  3.40s/it]<class 'list'>
this is i = 2
2
this is i = 4
6
this is i = 1
7
this is i = 3
10
this is i = 2
12
average = 0.24

